In [1]:
import os 
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from datetime import datetime
from pytz import timezone
from dateutil import tz
from datetime import datetime, timedelta
import geojson
import geopandas as gpd  
from fiona.crs import from_epsg
import os, json
from shapely.geometry import shape, Point, Polygon, MultiPoint

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
%matplotlib inline
import matplotlib.pyplot as plt
import osmnx as ox

from geopandas.tools import sjoin
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import train_test_split
import matplotlib.cm as cm
import folium

In [2]:
def getDuplicateColumns(frame):
    groups = frame.columns.to_series().groupby(frame.dtypes).groups
    dups = []
    for t, v in groups.items():
        dcols = frame[v].to_dict(orient="list")
        
        vs = list(dcols.values())
        ks = list(dcols.keys())
        lvs = len(vs)
        
        for i in range(lvs):
            vsi = vs[i]
            for j in range(i+1, lvs):
                vsj = vs[j]
                if vsi == vsj: 
                    dups.append(ks[i])
                    break
                    
    return dups

In [4]:
df_bruxelles = gpd.read_file('Anderlecht_streets.json')
print('Anderlecht total number of streets '+str(df_bruxelles.shape[0]))


polygons = df_bruxelles
m = folium.Map([50.85045, 4.34878], zoom_start=13, tiles='cartodbpositron')
folium.GeoJson(polygons).add_to(m)

Anderlecht total number of streets 2883


In [4]:
DF_5 = pd.read_csv('And_05min_0101_0103_2019.csv', header=None)
DF_5.columns = ['datetime','street_id','count','vel']
nRow_5, nCol_5 = DF_5.shape

DF_15 = pd.read_csv('And_15min_0101_0103_2019.csv', header=None)
DF_15.columns = ['datetime','street_id','count','vel']
nRow_15, nCol_15 = DF_15.shape

DF_30 = pd.read_csv('And_30min_0101_0103_2019.csv', header=None)
DF_30.columns = ['datetime','street_id','count','vel']
nRow_30, nCol_30 = DF_30.shape


# print(f'in Anderlecht 1 min there are {nRow_1} rows and {nCol_1} columns')
print(f'in Anderlecht 5 min there are {nRow_5} rows and {nCol_5} columns')
print(f'in Anderlecht 15 min there are {nRow_15} rows and {nCol_15} columns')
print(f'in Anderlecht 30 min there are {nRow_30} rows and {nCol_30} columns')

in Anderlecht 5 min there are 3364200 rows and 4 columns
in Anderlecht 15 min there are 2120983 rows and 4 columns
in Anderlecht 30 min there are 1592229 rows and 4 columns


In [5]:
table_5 = DF_5.pivot_table(index='datetime', columns='street_id')['count']
table_5 = table_5.fillna(0)

table_vel_5 = DF_5.pivot_table(index='datetime', columns='street_id')['vel']
table_vel_5 = table_vel_5.fillna(0)

print(table_5.shape)
print('')

table_15 = DF_15.pivot_table(index='datetime', columns='street_id')['count']
table_15 = table_15.fillna(0)

table_vel_15 = DF_15.pivot_table(index='datetime', columns='street_id')['vel']
table_vel_15 = table_vel_15.fillna(0)

print(table_15.shape)
print('')

table_30 = DF_30.pivot_table(index='datetime', columns='street_id')['count']
table_30 = table_30.fillna(0)

table_vel_30 = DF_30.pivot_table(index='datetime', columns='street_id')['vel']
table_vel_30 = table_vel_30.fillna(0)

print(table_30.shape)
print('')


(16894, 2852)

(5656, 2852)

(2832, 2852)



In [6]:
list_duplicates = getDuplicateColumns(table_vel_5)
print(len(list_duplicates))

778


In [7]:
file_name = 'Flow_Anderlecht_street_5min'
print(file_name)
table_5 = table_5.reset_index().drop(list_duplicates, axis=1)
table_5.to_csv(file_name + '.csv',index=False)
print(table_5.shape)
print('')

file_name = 'Velocity_Anderlecht_street_5min'
print(file_name)
table_vel_5 = table_vel_5.reset_index().drop(list_duplicates, axis=1)
table_vel_5.to_csv(file_name + '.csv',index=False)
print(table_vel_5.shape)
print('')
print('')


file_name = 'Flow_Anderlecht_street_15min'
print(file_name)
table_15 = table_15.reset_index().drop(list_duplicates, axis=1)
table_15.to_csv(file_name + '.csv',index=False)
print(table_15.shape)
print('')

file_name = 'Velocity_Anderlecht_street_15min'
print(file_name)
table_vel_15 = table_vel_15.reset_index().drop(list_duplicates, axis=1)
table_vel_15.to_csv(file_name + '.csv',index=False)
print(table_vel_15.shape)
print('')
print('')


file_name = 'Flow_Anderlecht_street_30min'
print(file_name)
table_30 = table_30.reset_index().drop(list_duplicates, axis=1)
table_30.to_csv(file_name + '.csv',index=False)
print(table_30.shape)
print('')

file_name = 'Velocity_Anderlecht_street_30min'
print(file_name)
table_vel_30 = table_vel_30.reset_index().drop(list_duplicates, axis=1)
table_vel_30.to_csv(file_name + '.csv',index=False)
print(table_vel_30.shape)
print('')
print('')

Flow_Anderlecht_street_5min
(16894, 2075)

Velocity_Anderlecht_street_5min
(16894, 2075)


Flow_Anderlecht_street_15min
(5656, 2075)

Velocity_Anderlecht_street_15min
(5656, 2075)


Flow_Anderlecht_street_30min
(2832, 2075)

Velocity_Anderlecht_street_30min
(2832, 2075)




In [5]:
df_bruxelles = gpd.read_file('Bruxelles_streets.json')
print('Bruxelles total number of streets '+str(df_bruxelles.shape[0]))


polygons = df_bruxelles
m = folium.Map([50.85045, 4.34878], zoom_start=12, tiles='cartodbpositron')
folium.GeoJson(polygons).add_to(m)

Bruxelles total number of streets 4540


In [9]:
DF_5 = pd.read_csv('Bxl_05min_0101_0103_2019.csv', header=None)
DF_5.columns = ['datetime','street_id','count','vel']
nRow_5, nCol_5 = DF_5.shape

DF_15 = pd.read_csv('Bxl_15min_0101_0103_2019.csv', header=None)
DF_15.columns = ['datetime','street_id','count','vel']
nRow_15, nCol_15 = DF_15.shape

DF_30 = pd.read_csv('Bxl_30min_0101_0103_2019.csv', header=None)
DF_30.columns = ['datetime','street_id','count','vel']
nRow_30, nCol_30 = DF_30.shape



print(f'in BXL 5 min there are {nRow_5} rows and {nCol_5} columns')
print(f'in BXL 15 min there are {nRow_15} rows and {nCol_15} columns')
print(f'in BXL 30 min there are {nRow_30} rows and {nCol_30} columns')

in BXL 5 min there are 11037714 rows and 4 columns
in BXL 15 min there are 6652675 rows and 4 columns
in BXL 30 min there are 4402294 rows and 4 columns


In [10]:
table_5 = DF_5.pivot_table(index='datetime', columns='street_id')['count']
table_5 = table_5.fillna(0)

table_vel_5 = DF_5.pivot_table(index='datetime', columns='street_id')['vel']
table_vel_5 = table_vel_5.fillna(0)

print(table_5.shape)
print('')


table_15 = DF_15.pivot_table(index='datetime', columns='street_id')['count']
table_15 = table_15.fillna(0)

table_vel_15 = DF_15.pivot_table(index='datetime', columns='street_id')['vel']
table_vel_15 = table_vel_15.fillna(0)

print(table_15.shape)
print('')

table_30 = DF_30.pivot_table(index='datetime', columns='street_id')['count']
table_30 = table_30.fillna(0)

table_vel_30 = DF_30.pivot_table(index='datetime', columns='street_id')['vel']
table_vel_30 = table_vel_30.fillna(0)

print(table_30.shape)
print('')


(16986, 4529)

(5664, 4529)

(2832, 4529)



In [11]:
list_duplicates = getDuplicateColumns(table_vel_5)
print(len(list_duplicates))

354


In [12]:
file_name = 'Flow_BXL_street_5min'
print(file_name)
table_5 = table_5.reset_index().drop(list_duplicates, axis=1)
table_5.to_csv(file_name + '.csv',index=False)
print(table_5.shape)
print('')

file_name = 'Velocity_BXL_street_5min'
print(file_name)
table_vel_5 = table_vel_5.reset_index().drop(list_duplicates, axis=1)
table_vel_5.to_csv(file_name + '.csv',index=False)
print(table_vel_5.shape)
print('')
print('')


file_name = 'Flow_BXL_street_15min'
print(file_name)
table_15 = table_15.reset_index().drop(list_duplicates, axis=1)
table_15.to_csv(file_name + '.csv',index=False)
print(table_15.shape)
print('')

file_name = 'Velocity_BXL_street_15min'
print(file_name)
table_vel_15 = table_vel_15.reset_index().drop(list_duplicates, axis=1)
table_vel_15.to_csv(file_name + '.csv',index=False)
print(table_vel_15.shape)
print('')
print('')


file_name = 'Flow_BXL_street_30min'
print(file_name)
table_30 = table_30.reset_index().drop(list_duplicates, axis=1)
table_30.to_csv(file_name + '.csv',index=False)
print(table_30.shape)
print('')

file_name = 'Velocity_BXL_street_30min'
print(file_name)
table_vel_30 = table_vel_30.reset_index().drop(list_duplicates, axis=1)
table_vel_30.to_csv(file_name + '.csv',index=False)
print(table_vel_30.shape)
print('')
print('')

Flow_BXL_street_5min
(16986, 4176)

Velocity_BXL_street_5min
(16986, 4176)


Flow_BXL_street_15min
(5664, 4176)

Velocity_BXL_street_15min
(5664, 4176)


Flow_BXL_street_30min
(2832, 4176)

Velocity_BXL_street_30min
(2832, 4176)




In [6]:
df_belgium = gpd.read_file('Belgium_streets.json')
print('Belgium total number of highways '+str(df_belgium.shape[0]))

m = folium.Map([50.85045, 4.34878], zoom_start=9, tiles='cartodbpositron')
folium.GeoJson(df_belgium).add_to(m)

Belgium total number of highways 5197


In [14]:
DF_5 = pd.read_csv('Bel_05min_0101_0103_2019.csv', header=None)
DF_5.columns = ['datetime','street_id','count','vel']
nRow_5, nCol_5 = DF_5.shape

DF_15 = pd.read_csv('Bel_15min_0101_0103_2019.csv', header=None)
DF_15.columns = ['datetime','street_id','count','vel']
nRow_15, nCol_15 = DF_15.shape

DF_30 = pd.read_csv('Bel_30min_0101_0103_2019.csv', header=None)
DF_30.columns = ['datetime','street_id','count','vel']
nRow_30, nCol_30 = DF_30.shape

DF_60 = pd.read_csv('Bel_60min_0101_0103_2019.csv', header=None)
DF_60.columns = ['datetime','street_id','count','vel']
nRow_60, nCol_60 = DF_60.shape

# print(f'in BEL 5 min there are {nRow_5} rows and {nCol_5} columns')
print(f'in BEL 15 min there are {nRow_15} rows and {nCol_15} columns')
print(f'in BEL 30 min there are {nRow_30} rows and {nCol_30} columns')
print(f'in BEL 60 min there are {nRow_60} rows and {nCol_60} columns')

in BEL 15 min there are 22657292 rows and 4 columns
in BEL 30 min there are 11245888 rows and 4 columns
in BEL 60 min there are 7826424 rows and 4 columns


In [15]:
table_5 = DF_5.pivot_table(index='datetime', columns='street_id')['count']
table_5 = table_5.fillna(0)

table_vel_5 = DF_5.pivot_table(index='datetime', columns='street_id')['vel']
table_vel_5 = table_vel_5.fillna(0)

table_15 = DF_15.pivot_table(index='datetime', columns='street_id')['count']
table_15 = table_15.fillna(0)

table_vel_15 = DF_15.pivot_table(index='datetime', columns='street_id')['vel']
table_vel_15 = table_vel_15.fillna(0)

table_30 = DF_30.pivot_table(index='datetime', columns='street_id')['count']
table_30 = table_30.fillna(0)

table_vel_30 = DF_30.pivot_table(index='datetime', columns='street_id')['vel']
table_vel_30 = table_vel_30.fillna(0)

table_60 = DF_60.pivot_table(index='datetime', columns='street_id')['count']
table_60 = table_60.fillna(0)

table_vel_60 = DF_60.pivot_table(index='datetime', columns='street_id')['vel']
table_vel_60 = table_vel_60.fillna(0)

print(table_60.shape)
print('')

(1416, 5195)



In [16]:
list_duplicates = getDuplicateColumns(table_vel_5)
print(len(list_duplicates))

22


In [17]:
file_name = 'Flow_BEL_street_5min'
print(file_name)
table_5 = table_5.reset_index().drop(list_duplicates, axis=1)
table_5.to_csv(file_name + '.csv',index=False)
print(table_5.shape)
print('')

file_name = 'Velocity_BEL_street_5min'
print(file_name)
table_vel_5 = table_vel_5.reset_index().drop(list_duplicates, axis=1)
table_vel_5.to_csv(file_name + '.csv',index=False)
print(table_vel_5.shape)
print('')
print('')


file_name = 'Flow_BEL_street_15min'
print(file_name)
table_15 = table_15.reset_index().drop(list_duplicates, axis=1)
table_15.to_csv(file_name + '.csv',index=False)
print(table_15.shape)
print('')

file_name = 'Velocity_BEL_street_15min'
print(file_name)
table_vel_15 = table_vel_15.reset_index().drop(list_duplicates, axis=1)
table_vel_15.to_csv(file_name + '.csv',index=False)
print(table_vel_15.shape)
print('')
print('')


file_name = 'Flow_BEL_street_30min'
print(file_name)
table_30 = table_30.reset_index().drop(list_duplicates, axis=1)
table_30.to_csv(file_name + '.csv',index=False)
print(table_30.shape)
print('')

file_name = 'Velocity_BEL_street_30min'
print(file_name)
table_vel_30 = table_vel_30.reset_index().drop(list_duplicates, axis=1)
table_vel_30.to_csv(file_name + '.csv',index=False)
print(table_vel_30.shape)
print('')
print('')


file_name = 'Flow_BEL_street_60min'
print(file_name)
table_60 = table_60.reset_index().drop(list_duplicates, axis=1)
table_60.to_csv(file_name + '.csv',index=False)
print(table_60.shape)
print('')

file_name = 'Velocity_BEL_street_60min'
print(file_name)
table_vel_60 = table_vel_60.reset_index().drop(list_duplicates, axis=1)
table_vel_60.to_csv(file_name + '.csv',index=False)
print(table_vel_60.shape)

Flow_BEL_street_5min
(16992, 5174)

Velocity_BEL_street_5min
(16992, 5174)


Flow_BEL_street_15min
(5664, 5174)

Velocity_BEL_street_15min
(5664, 5174)


Flow_BEL_street_30min
(2832, 5174)

Velocity_BEL_street_30min
(2832, 5174)


Flow_BEL_street_60min
(1416, 5174)

Velocity_BEL_street_60min
(1416, 5174)
